### @property装饰器

#### 之前我们讨论过python中属性和方法权限的问题，虽然不建议将属性设置为私有的，但是如果直接将属性暴露给外界也是有问题的，比如我们没有办法检查赋给属性的值是否有效。建议将属性命名以单下划线开头，通过这种方式来暗示属性是受保护的，不建议外界直接访问，那么外界直接访问属性可以通过属性的getter（访问器）和setter（修改器）方法进行对应的操作，如果要做到这点，就要考虑使用@property装饰器来包装getter和setter方法，是的属性的访问既安全有方便，代码如下。

In [4]:
class Person(object):
    def __init__(self,name,age):
        self._name = name
        self._age = age
        
    #访问器 - getter方法
    @property
    def name(self):
        return self._name
    
    #访问器 - getter方法
    @property
    def age(self):
        return self._age
    
    #修改器 - setter方法
    @age.setter
    def age(self,age):
        self._age = age
        
    def  play(self):
        if self._age <= 16:
            print('%s正在玩飞行棋。'% self._name)
        else:
            print('%s正在玩斗地主。'% self._name)
            
            
def main():
    person = Person('Kevin',12)
    person.play()
    person.age = 22
    person.play()
    
    
if __name__ == "__main__":
    main()

Kevin正在玩飞行棋。
Kevin正在玩斗地主。


### __slots__魔法

#### 所以说python是一门动态语言。通常动态语言允许我们在程序运行时给对象绑定新的属性或是方法，当然也可以读绑定的属性和方法进行解绑。但是如果我们需要限定自定义类型的对象只能绑定某些属性，可以通过在类中蒂尼__slots__变量来进行限定。需要注意的是__slots__的限定支队当前类的的对象生效，对子类并不起任何作用。

In [9]:
class Person(object):
    #限定Person对象只能绑定_name,_age,_gender属性
    __slots__ = ('_name','_age','_gender')
     
    def __init__(self,name,age):
        self._name = name
        self._age = age
        
    @property
    def name(self):
        return self._name
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self,age):
        self._age = age
        
    def play(self):
        if self._age <= 16:
            print('%s正在玩飞行棋。'%self._name)
        else:
            print('%s正在斗地主.' % self._name)
            
def main():
    person = Person('Kevin',22)
    person.play()
    person._gender = '男'
    person._is_gay = True # AttributeError: 'Person' object has no attribute '_is_gay'
    
if __name__ == "__main__":
    main()
    

Kevin正在斗地主.


AttributeError: 'Person' object has no attribute '_is_gay'

### 静态方法和类方法

#### 之前，我们在类中定义的方法都是对象方法，也就是说这些方法都是发送给对象的消息。实际上，我们在写类的方法并不需要都是对象方法。例如定义一个"三角形"类，通过传入三条边来构造三角形，并提供计算周长和面积的方法，但是传入的三条边未必能构造出三角形对象，因此我们可以先写出一个方法来验证三条边是否可以构造三角形，合格方法显然就不是对象方法，因为在调用这个方法是三角形对象尚未创建出来，所以这个方法是属于三角形类而不是三角形对象。我们可以使用静态方法来解决这类问题，代码如下。

In [13]:
from math import sqrt

class Triangle(object):
    def __init__(self,a,b,c):
        self._a = a 
        self._b = b
        self._c = c
        
    @staticmethod
    def is_valid(a,b,c):
        return a + b > c and a + c > b and b + c > a
    
    def  perimeter(self):
        return self._a+self._b+self._c
    
    def area(self):
        half = self.perimeter() / 2
        return sqrt(half * (half - self._a) * (half - self._b) * (half - self._c))
    
def main():
    a,b,c = 3,4,5
    #静态方法和类方法都是通过给类发消息来调用的
    if Triangle.is_valid(a,b,c):
        t = Triangle(a,b,c)
        print('三角形的周长为：%.2f'% t.perimeter())
        #也可以通过给类发消息来调用对象方法但是要传入接受消息的对象做参数
        #print(Triangle.perimeter(t))
        print('三角形的面积为：%.2f' % t.area())
        #print(Triangle.area(t))
    else:
        print('无法构成三角形。')
        
if __name__ == "__main__":
    main()


三角形的周长为：12.00
三角形的面积为：6.00


#### 和静态方法比较类似，python还可以在类中定义类的方法，类方法的第一个参数约定名为cls，他代表的是当前类相关的信息的对象（类本身也是一个对象，有的地方也称之类的元数据对象），通过这个参数我们可以获取类相关的心绪并且可以创建出类的对象，代码如下。

In [14]:
from time import time,localtime,sleep

class Clock(object):
    """数字时钟"""
    
    def __init__(self,hour=0,minute=0,second=0):
        self._hour = 0 
        self._minute = 0
        self._second = 0
        
    @classmethod
    def now(cls):
        ctime = localtime(time())
        return cls(ctime.tm_hour,ctime.tm_min,ctime.tm_sec)
    
    def run(self):
        """走字"""
        
        self._second += 1 
        if self._second == 60:
            self._minute += 1
            self._second = 0
            if self._minute == 60:
                self._hour += 1
                self._minute = 0
                if  self._hour == 24:
                    self._hour = 0
                    
    def show(self):
        """显示时间"""
        
        return '%2d:%2d:%2d' % (self._hour,self._minute,self._second)
    
def main():
    #通过类方法创建对象并获取系统时间
    clock = Clock()
    while True:
        print(clock.show())
        sleep(1)
        clock.run()
        
        
if __name__ == "__main__":
    main()

 0: 0: 0
 0: 0: 1
 0: 0: 2
 0: 0: 3
 0: 0: 4
 0: 0: 5
 0: 0: 6
 0: 0: 7
 0: 0: 8


KeyboardInterrupt: 

### 类之间的关系

#### 简单的说，类和类之间的关系有三种：is-a、has-a和use-a关系。
#### is-a关系也叫继承或泛化，比如学生和人的关系、手机和电子产品的关系属于继承关系
#### has-a关系称之为关联，比如部门和员工的关系，汽车和引擎的关系
#### use-a称之为依赖，比如司机有一个驾驶的行为（方法），其中（的参数）使用到了汽车，那么汽车和司机的关系就是依赖关系

#### 利用类之间的这些关系，可以在已有的类的基础上来完成某些操作，也可以在已有的类的基础上创建新的类，这些都是事先代码复用的重要手段。复用现有的代码不仅可以减少开发的工作量，也有利于代码的管理和维护。

### 继承和多态

#### 刚才我们提到了，可以在已有类的基础上创建新类，这其中的一种做法就是让一个类从另一个类那里将属性和方法直接继承下来，从而减少重复代码的编写。提供继承信息的我们称之为父类，也叫超类或基类；得到继承信息的我们称之为子类，也叫派生类或衍生类。子类除了继承父类提供的属性和方法，还可以定义自己特有的属性和方法，所以子类比父类拥有的更多的能力，在实际开发中，我们经常会用子类对象去替换掉一个父类对象，这是面向对象编程中一个常见的行为，对应的原则称之为里氏替换原则。

In [19]:
class Person(object):
    """人"""
    
    def __init__(self,name,age):
        self._name = name
        self._age = age
        
    @property
    def name(self):
        return self._name
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self,age):
        self._age = age
        
    def play(self):
        print('%s正在愉快的玩耍。' % self._name)
        
    def watch_av(self):
        if self._age <= 18:
            print('%s正在观看熊出没。' % self._name)
        else:
            print('%s正在观看爱情动作片。' % self._name)
            
class Student(Person):
    """学生"""
    
    def __init__(self,name,age,grade):
        super().__init__(name,age)
        self._grade = grade
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self,grade):
        self._grade = grade
        
    def study(self,course):
        print('%s的%s正在学习%s。'% (self._grade,self._name,course))
        
class Teacher(Person):
    """老师"""
    
    def __init__(self,name,age,title):
        super().__init__(name,age)
        self._title = title
        
    @property
    def title(self):
        return self._title
    
    @title.setter
    def title(self,title):
        self._title = title
        
    def teach(self,course):
        print('%s%s正在讲%s.' % (self._name,self._title,course))
        
def main():
    stu = Student('Kevin',15,'初一')
    stu.study('数学')
    stu.watch_av()
    t = Teacher('Kerr',45,'Coach')
    t.teach('basketball')
    t.watch_av()
    
if __name__ == "__main__":
    main()

初一的Kevin正在学习数学。
Kevin正在观看熊出没。
KerrCoach正在讲basketball.
Kerr正在观看爱情动作片。


#### 子类在继承父类的方法后，可以对父类已有的方法给出新的实现版本，这个动作称之为方法重写（override）。通过方法重写我们可以让父类的同一个行为在子类中拥有不同的实现版本，当我们调用这个经过子类重写的方法是，不同的子类对象会表现出不同的行为，这个就是多态（poly-morphism）

In [22]:
from abc import ABCMeta,abstractclassmethod

class Pet(object,metaclass=ABCMeta):
    """宠物"""
    def __init__(self,nickname):
        self._nickname = nickname
        
    @abstractclassmethod
    def make_voice(self):
        """发出声音"""
        pass
    
class Dog(Pet):
    """狗"""
    
    def make_voice(self):
        print('%s:汪汪汪...' % self._nickname)
        
class Cat(Pet):
    """猫"""
    
    def make_voice(self):
        print('%s:喵喵喵...' % self._nickname)
        
def main():
    pets = [Dog('旺财'),Cat('凯蒂'),Dog('大黄')]
    for pet in pets:
        pet.make_voice()
        
if __name__ == "__main__":
    main()

旺财:汪汪汪...
凯蒂:喵喵喵...
大黄:汪汪汪...


#### 在上面的代码中，我们将Pet类处理成了一个抽象类，所谓抽象类就是不能够创建对象的类，这种类的存在就是专门为了让其他类去继承它。Python从语法层面并没有像Java或C#那样提供对抽象类的支持，但是我们可以通过abc模块的ABCMeta元类和abstractmethod包装器来达到抽象类的效果，如果一个类中存在抽象方法那么这个类就不能够实例化（创建对象）。上面的代码中，Dog和Cat两个子类分别对Pet类中的make_voice抽象方法进行了重写并给出了不同的实现版本，当我们在main函数中调用该方法时，这个方法就表现出了多态行为（同样的方法做了不同的事情）。

In [30]:
# 练习1、奥特曼打小怪兽
from abc import ABCMeta, abstractmethod
from random import randint, randrange


class Fighter(object, metaclass=ABCMeta):
    """战斗者"""

    # 通过__slots__魔法限定对象可以绑定的成员变量
    __slots__ = ('_name', '_hp')

    def __init__(self, name, hp):
        """
        初始化方法

        :param name: 名字
        :param hp: 生命值
        """
        self._name = name
        self._hp = hp

    @property
    def name(self):
        return self._name

    @property
    def hp(self):
        return self._hp

    @hp.setter
    def hp(self, hp):
        self._hp = hp if hp >= 0 else 0

    @property
    def alive(self):
        return self._hp > 0

    @abstractmethod
    def attack(self, other):
        """
        攻击

        :param other: 被攻击的对象
        """
        pass


class Ultraman(Fighter):
    """奥特曼"""

    __slots__ = ('_name', '_hp', '_mp')

    def __init__(self, name, hp, mp):
        """
        初始化方法

        :param name: 名字
        :param hp: 生命值
        :param mp: 魔法值
        """
        super().__init__(name, hp)
        self._mp = mp

    def attack(self, other):
        other.hp -= randint(15, 25)

    def huge_attack(self, other):
        """
        究极必杀技(打掉对方至少50点或四分之三的血)

        :param other: 被攻击的对象

        :return: 使用成功返回True否则返回False
        """
        if self._mp >= 50:
            self._mp -= 50
            injury = other.hp * 3 // 4
            injury = injury if injury >= 50 else 50
            other.hp -= injury
            return True
        else:
            self.attack(other)
            return False

    def magic_attack(self, others):
        """
        魔法攻击

        :param others: 被攻击的群体

        :return: 使用魔法成功返回True否则返回False
        """
        if self._mp >= 20:
            self._mp -= 20
            for temp in others:
                if temp.alive:
                    temp.hp -= randint(10, 15)
            return True
        else:
            return False

    def resume(self):
        """恢复魔法值"""
        incr_point = randint(1, 10)
        self._mp += incr_point
        return incr_point

    def __str__(self):
        return '~~~%s奥特曼~~~\n' % self._name + \
            '生命值: %d\n' % self._hp + \
            '魔法值: %d\n' % self._mp


class Monster(Fighter):
    """小怪兽"""

    __slots__ = ('_name', '_hp')

    def attack(self, other):
        other.hp -= randint(10, 20)

    def __str__(self):
        return '~~~%s小怪兽~~~\n' % self._name + \
            '生命值: %d\n' % self._hp


def is_any_alive(monsters):
    """判断有没有小怪兽是活着的"""
    for monster in monsters:
        if monster.alive > 0:
            return True
    return False


def select_alive_one(monsters):
    """选中一只活着的小怪兽"""
    monsters_len = len(monsters)
    while True:
        index = randrange(monsters_len)
        monster = monsters[index]
        if monster.alive > 0:
            return monster


def display_info(ultraman, monsters):
    """显示奥特曼和小怪兽的信息"""
    print(ultraman)
    for monster in monsters:
        print(monster, end='')


def main():
    u = Ultraman('骆昊', 1000, 120)
    m1 = Monster('舒小玲', 250)
    m2 = Monster('白元芳', 500)
    m3 = Monster('王大锤', 750)
    ms = [m1, m2, m3]
    fight_round = 1
    while u.alive and is_any_alive(ms):
        print('========第%02d回合========' % fight_round)
        m = select_alive_one(ms)  # 选中一只小怪兽
        skill = randint(1, 10)   # 通过随机数选择使用哪种技能
        if skill <= 6:  # 60%的概率使用普通攻击
            print('%s使用普通攻击打了%s.' % (u.name, m.name))
            u.attack(m)
            print('%s的魔法值恢复了%d点.' % (u.name, u.resume()))
        elif skill <= 9:  # 30%的概率使用魔法攻击(可能因魔法值不足而失败)
            if u.magic_attack(ms):
                print('%s使用了魔法攻击.' % u.name)
            else:
                print('%s使用魔法失败.' % u.name)
        else:  # 10%的概率使用究极必杀技(如果魔法值不足则使用普通攻击)
            if u.huge_attack(m):
                print('%s使用究极必杀技虐了%s.' % (u.name, m.name))
            else:
                print('%s使用普通攻击打了%s.' % (u.name, m.name))
                print('%s的魔法值恢复了%d点.' % (u.name, u.resume()))
        if m.alive > 0:  # 如果选中的小怪兽没有死就回击奥特曼
            print('%s回击了%s.' % (m.name, u.name))
            m.attack(u)
        display_info(u, ms)  # 每个回合结束后显示奥特曼和小怪兽的信息
        fight_round += 1
    print('\n========战斗结束!========\n')
    if u.alive > 0:
        print('%s奥特曼胜利!' % u.name)
    else:
        print('小怪兽胜利!')


if __name__ == '__main__':
    main()


========第01回合========
骆昊使用普通攻击打了白元芳.
骆昊的魔法值恢复了4点.
白元芳回击了骆昊.
~~~骆昊奥特曼~~~
生命值: 988
魔法值: 124

~~~舒小玲小怪兽~~~
生命值: 250
~~~白元芳小怪兽~~~
生命值: 481
~~~王大锤小怪兽~~~
生命值: 750
========第02回合========
骆昊使用普通攻击打了王大锤.
骆昊的魔法值恢复了10点.
王大锤回击了骆昊.
~~~骆昊奥特曼~~~
生命值: 976
魔法值: 134

~~~舒小玲小怪兽~~~
生命值: 250
~~~白元芳小怪兽~~~
生命值: 481
~~~王大锤小怪兽~~~
生命值: 734
========第03回合========
骆昊使用了魔法攻击.
王大锤回击了骆昊.
~~~骆昊奥特曼~~~
生命值: 958
魔法值: 114

~~~舒小玲小怪兽~~~
生命值: 239
~~~白元芳小怪兽~~~
生命值: 468
~~~王大锤小怪兽~~~
生命值: 723
========第04回合========
骆昊使用了魔法攻击.
王大锤回击了骆昊.
~~~骆昊奥特曼~~~
生命值: 944
魔法值: 94

~~~舒小玲小怪兽~~~
生命值: 229
~~~白元芳小怪兽~~~
生命值: 453
~~~王大锤小怪兽~~~
生命值: 709
========第05回合========
骆昊使用普通攻击打了舒小玲.
骆昊的魔法值恢复了8点.
舒小玲回击了骆昊.
~~~骆昊奥特曼~~~
生命值: 929
魔法值: 102

~~~舒小玲小怪兽~~~
生命值: 211
~~~白元芳小怪兽~~~
生命值: 453
~~~王大锤小怪兽~~~
生命值: 709
========第06回合========
骆昊使用了魔法攻击.
舒小玲回击了骆昊.
~~~骆昊奥特曼~~~
生命值: 909
魔法值: 82

~~~舒小玲小怪兽~~~
生命值: 198
~~~白元芳小怪兽~~~
生命值: 442
~~~王大锤小怪兽~~~
生命值: 696
========第07回合========
骆昊使用普通攻击打了王大锤.
骆昊的魔法值恢复了9点.
王大锤回击了骆昊.
~~~骆昊奥特曼~~~
生命值: 889
魔法值: 91

~~~舒小玲小怪兽~~~
生命值: 19